<font face="Times New Roman" size=6 >
GE5219 Project

# Facebook Movement between Tiles

## Source Code

In [ ]:
import pandas as pd
import math
import os
import datetime
from datetime import time
import re

### Read File

In [ ]:
filepath = r"E:\GRADUATE STUDY\GE5219\Data\Trial\movement\2662837396048326_2020-03-26_1600.csv"
df = pd.read_csv(filepath)
df

### Merge File

In [ ]:
# set up variables
path = 'E://GRADUATE STUDY//GE5219//Data//Movement//202003'  # Folder Path
filename_extenstion = '.csv'  # File extension
file_allname = []  # Store all file names ready for merge
cols_new_name = ['geometry', 'date_time', 'length_km', 'country', 'n_crisis','baseline','difference','percentage','z_score','start_lat','start_lon','end_lat','end_lon']
cols_num = [0, 1, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18, 19]  # Index of columns that are ready for merge, starting from 0


# list file names
for filename in os.listdir(path): # look for files in the folder
    if os.path.splitext(filename)[1] == filename_extenstion:
        t = os.path.splitext(filename)[0]
        file_allname.append(path + '/' + t + filename_extenstion)  # restore csv suffix
        
df = pd.DataFrame(cols_new_name).T
df.to_csv("temp.csv", header=False, index=False)
for fn in file_allname:
    data = pd.read_csv(fn)
    data = data.iloc[1:, cols_num]
    data.to_csv("temp.csv", mode='a', header=False, index=False)

# Auto Rename
os.rename("temp.csv",data.iloc[1,1].split("-")[0] + data.iloc[1,1].split("-")[1] + ".csv")
print('Merge Complete')


### Remove Rebundant Countries

In [ ]:
filepath = "202003.csv"
df1 = pd.read_csv(filepath)
df2 = df1.loc[(df1['country'] == 'SG') | (df1['country'] == 'VN')]

# AGGREGATE BY DAY
aggregate_column = ['length_km','n_crisis']

# Split date and time
date = df2['date_time'].str.split('-', expand=True)
time = date[2].str.split(' ', expand=True)
split_data = pd.concat([df2, date[[0,1]], time], axis = 1)

# Fill in NaN
split_data.loc[:,['n_crisis','baseline']] = split_data.loc[:,['n_crisis','baseline']].fillna(5)

# Rename columns
def df_column_uniquify(df):
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 0
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df

df_column_uniquify(split_data)

split_data.rename(columns={0:"year",1:"month","0_1":"day","1_1":"time"}, inplace=True)

# Aggregate by time
aggregate_value = ['n_crisis','difference','baseline']
aggregate = split_data.groupby(['country','start_lat','start_lon','end_lat','end_lon','year','month','day'])[aggregate_value].sum()
aggregate = aggregate.reset_index() # return dataframe of groupby result

# CALCULATE which GRID the trajectory is in, the grid index starts from 0
startgrid_x = []
startgrid_y = []
endgrid_x = []
endgrid_y = []
dif_x = []
dif_y = []

for index, row in aggregate.iterrows():
    startgridx = math.floor((row['start_lon'] - 92.25)/0.25)
    startgridy = math.floor((28.5 - row['start_lat'])/0.25)
    endgridx = math.floor((row['end_lon'] - 92.25)/0.25)
    endgridy = math.floor((28.5 - row['end_lat'])/0.25)
    difx = abs(startgridx - endgridx)
    dify = abs(startgridy - endgridy)
    
    startgrid_x.append(startgridx)
    startgrid_y.append(startgridy)
    endgrid_x.append(endgridx)
    endgrid_y.append(endgridy)
    dif_x.append(difx)
    dif_y.append(dify)

grid = pd.DataFrame({"startgrid_x":startgrid_x,
                    "startgrid_y":startgrid_y,
                    "endgrid_x":endgrid_x,
                    "endgrid_y":endgrid_y,
                    "dif_x":dif_x,
                    "dif_y":dif_y})

with_grid = pd.concat([aggregate, grid], axis = 1)

# Filter those data not within one or two adjacent grid
with_grid.drop(with_grid[(with_grid['dif_x']>1) | (with_grid['dif_y']>1)].index,axis=0, inplace = True)

# AGGREGATE BY GRID
aggregate_value = ['n_crisis','difference','baseline']
aggregate2 = with_grid.groupby(['country','year','month','day','startgrid_x','startgrid_y','endgrid_x','endgrid_y'])[aggregate_value].sum()
aggregate2 = aggregate2.reset_index() # return dataframe of groupby result
date = aggregate2['year'].str.cat(aggregate2['month'],sep='-').str.cat(aggregate2['day'],sep='-')
aggregate2 = aggregate2.drop(['year','month','day'], axis = 1)
aggregate2 = pd.concat([aggregate2, date], axis = 1)
aggregate2.to_csv("aggtemp.csv", mode='a', header=True, index=False)

# Auto rename
os.rename("aggtemp.csv","Agg" + aggregate2.iloc[0,1] + aggregate2.iloc[0,2] + ".csv")
print('Aggregate Complete')

### Assign Value to Grid

In [ ]:
filepath = "Agg202003.csv"
df3 = pd.read_csv(filepath)
df3.eval('movement_value = n_crisis / 2' , inplace=True)
df4 = df3.drop(["endgrid_x","endgrid_y"], axis = 1)
df5 = df3.drop(["startgrid_x","startgrid_y"], axis = 1)
df4.rename(columns={"startgrid_x":"grid_x", "startgrid_y":"grid_y"}, inplace = True)
df5.rename(columns={"endgrid_x":"grid_x", "endgrid_y":"grid_y"}, inplace = True)
Cal_grid = pd.concat([df4, df5], axis = 0)
Cal_grid = Cal_grid.reset_index()
Cal_grid.to_csv("gridtemp.csv", mode='a', header=True, index=False)

# Auto rename
os.rename("gridtemp.csv","Grid" + str(Cal_grid.iloc[0,2]) + str(Cal_grid.iloc[0,3]) + ".csv")
print('Generate Grid Complete')

### Merge Files From All Countries

In [ ]:
# set up variables
path = r'E:\GRADUATE STUDY\GE5219\Data\Movement\Results\Processed data'  # Folder Path
filename_extenstion = '.csv'  # File extension
cols_new_name = ["country","date","grid_x","grid_y","n_crisis","difference","baseline","movement_value","grid_lon","grid_lat"]  # column names after merging, modify according to needs

# list file names
for year in range(2020,2023):
    for month in range(1,13):
        if year == 2020 and month<3:
            continue
        elif year == 2022 and month>2:
            break
        else:
            file_allname = []  # Store all file names ready for merge
            for filename in os.listdir(path): # look for files in the folder
                if os.path.splitext(filename)[1] == filename_extenstion and "Grid" in filename and "{}{:0>2d}".format(year, month) in filename: 
                    t = os.path.splitext(filename)[0]
                    file_allname.append(path + '/' + t + filename_extenstion)  # restore csv suffix

            df = pd.DataFrame(cols_new_name).T
            df.to_csv("temp.csv", header=False, index=False) 

            for fn in file_allname:
                data = pd.read_csv(fn)
                data = data.iloc[1:,0:10]
                data.to_csv("temp.csv", mode='a', header=False, index=False)

            # auto rename
            os.rename("temp.csv","{}.{:0>2d}.csv".format(year,month))
            print("Merge Complete {}.{:0>2d}.csv".format(year,month))


### Drop Duplicates

In [ ]:
filepath = r"E:\GRADUATE STUDY\GE5219\Data\Study Area\Study Area Refined.csv"
dp = pd.read_csv(filepath)
dp2 = dp.drop_duplicates()
dp2

### For Case Study

In [ ]:
# Aggregate monthly data
folder_path = "E:\GRADUATE STUDY\GE5219\Data\Movement\Results\Processed data"
for filename in os.listdir(folder_path):
    if "GridID" in filename or "GridSG" in filename:
        df = pd.read_csv(folder_path + "\\" + filename, header = 0)
        df = df.groupby(['grid_lon','grid_lat','country']).mean(['n_crisis','difference','baseline','movement_value'])
        df = df.reset_index()
        df['date']= filename[6:12]
        df.to_csv(filename[4:], columns = ['grid_lon','grid_lat','movement_value','date','country'], index = 0)

In [ ]:
# Merge monthly country data
folder_path = "E:\GRADUATE STUDY\GE5219\Data\Movement\Results\Monthly"
SG = []
ID = []
for filename in os.listdir(folder_path):
    df = pd.read_csv(folder_path + "\\" + filename, header = 0)
    if "SG" in filename:
        SG.append(df)
    if "ID" in filename:
        ID.append(df)
SG = pd.concat(SG)
SG["country"].fillna(("SG"), inplace = True)
ID = pd.concat(ID)
ID["country"].fillna(("ID"), inplace = True)
Merge = pd.concat([ID,SG], axis = 0)
SG.to_csv('SG.csv', index = 0)
ID.to_csv('ID.csv')

In [ ]:
Merge['date'] = Merge['date'].map(lambda x: datetime.datetime.strptime(str(x), '%Y%m'))

In [ ]:
Merge.to_csv("Space-time Analysis.csv", index = 0)

In [ ]:
# Aggregate by month
folder_path = "E:\GRADUATE STUDY\GE5219\Data\Movement\Results\Exclude all months with abnormal value"

for filename in os.listdir(folder_path):
    df = pd.read_csv(folder_path + "\\" + filename, header = 0)
    if '-' in df['date']:
        date = df['date'].str.split('-', expand=False)
    elif '/' in df['date']:
        date = df['date'].str.split('/', expand=False)
    print(date)
    date = df['0'].str.cat(df['1'],sep='/')
    print(date)
#     split_df = pd.concat([df, date], axis = 1)
#     print(df)

In [ ]:

    aggregate = df.groupby(date.Grouper(freq='M'))['n_crisis','difference','baseline','movement_value'].sum()
#    aggregate = df.groupby(['country','grid_x','grid_y','grid_lon','grid_lat', pd.Grouper(freq='M')])['n_crisis','difference','baseline','movement_value'].sum()
    df = aggregate.reset_index() # return dataframe of groupby result
    date = aggregate2['year'].str.cat(aggregate2['month'],sep='-').str.cat(aggregate2['day'],sep='-')
    aggregate3 = aggregate2.drop(['year','month','day'], axis = 1)
    aggregate3 = pd.concat([aggregate3, date], axis = 1)
    aggregate3.rename(columns={"year":"date"}, inplace=True)
    
    print(df)
#     df.to_csv("temp.csv", mode = 'a', header=True, index=False)
#     os.rename("temp.csv",filename)

print('Complete')

## Function

### Make Folder Path

In [ ]:
def Make_File_Path(filepath, countries_list):
    file_path=os.path.abspath(filepath)
    for country in countries_list:
        for year in range(2020,2023):
            for month in range(1,13):
                if year==2020 and month<3:
                    continue
                elif year==2022 and month>2:
                    break
                else:
                    file = "{}{}{:0>2d}".format(country,year,month)
                    file_name=file_path + "\\"+file
                    os.makedirs(file_name)

In [ ]:
Make_File_Path("E:\GRADUATE STUDY\GE5219\Data\Movement",["Philippines","Myanmar","Cambodia","Brunei"])

### Process Mobility File

In [ ]:
def Process_File(folder_path, country):
    # MERGE FILE

    # set up variables
    filename_extenstion = '.csv'  # File extension
    file_allname = []  # Store all file names ready for merge
    cols_new_name = ['geometry', 'date_time', 'length_km', 'country', 'n_crisis','baseline','difference','percentage','z_score','start_lat','start_lon','end_lat','end_lon']  # 汇总后的列名，根据需要修改
    cols_num = [0, 1, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18, 19]  # Index of columns that are ready for merge, starting from 0


    # list file names
    for filename in os.listdir(folder_path): # look for files in the folder
        if os.path.splitext(filename)[1] == filename_extenstion:
            t = os.path.splitext(filename)[0]
            file_allname.append(folder_path + '/' + t + filename_extenstion)  # restore csv suffix

#    frame = pd.DataFrame(cols_new_name).T
    data = []
    
    for filenames in file_allname:
        df = pd.read_csv(filenames, header = 0)
        df = df.iloc[0:, cols_num]
        df.columns = df.columns.map(lambda x:x.lower())
        data.append(df) 

    data = pd.concat(data)
    data.columns = cols_new_name

#    CODE USED TO GENERATE RAW DATA CSV:
#    df = pd.DataFrame(cols_new_name).T
#    df.to_csv("temp.csv", header=False, index=False)
#    for fn in file_allname:
#       data = pd.read_csv(fn)
#        data = data.iloc[1:, cols_num]
#        data.to_csv("temp.csv", mode='a', header=False, index=False)

    # Auto Rename
#    os.rename("temp.csv",data.iloc[1,1].split("-")[0] + data.iloc[1,1].split("-")[1] + ".csv")
#    print('Merge Complete')
        
#    df1 = pd.read_csv(data.iloc[1,1].split("-")[0] + data.iloc[1,1].split("-")[1] + ".csv")
#    df2 = df1.loc[(df1['country'] == country)]

    # REMOVE REBUNDANT COUNTRIES
    df2 = data.loc[(data['country'] == country)]
    
    # AGGREGATE BY DAY
    # Split date and time
    date = df2['date_time'].str.split('-', expand=True)
    time = date[2].str.split(' ', expand=True)
    split_data = pd.concat([df2, date[[0,1]], time], axis = 1)

    # Fill in NaN
    split_data.loc[:,'n_crisis'] = split_data.loc[:,'n_crisis'].fillna(5)
    split_data.loc[:,'baseline'] = split_data.loc[:,'baseline'].fillna(5)
    
    # Rename columns
    def df_column_uniquify(df):
        df_columns = df.columns
        new_columns = []
        for item in df_columns:
            counter = 0
            newitem = item
            while newitem in new_columns:
                counter += 1
                newitem = "{}_{}".format(item, counter)
            new_columns.append(newitem)
        df.columns = new_columns
        return df

    df_column_uniquify(split_data)

    split_data.rename(columns={0:"year",1:"month","0_1":"day","1_1":"time"}, inplace=True)

    # Aggregate by time
    aggregate_value = ['n_crisis','difference','baseline']
    aggregate = split_data.groupby(['country','start_lat','start_lon','end_lat','end_lon','year','month','day'])[aggregate_value].sum()
    aggregate = aggregate.reset_index() # return dataframe of groupby result

    # CALCULATE which GRID the trajectory is in, the grid index starts from 0
    startgrid_x = []
    startgrid_y = []
    endgrid_x = []
    endgrid_y = []
    dif_x = []
    dif_y = []

    for index, row in aggregate.iterrows():
        startgridx = math.floor((row['start_lon'] - 92.25)/0.25)
        startgridy = math.floor((28.5 - row['start_lat'])/0.25)
        endgridx = math.floor((row['end_lon'] - 92.25)/0.25)
        endgridy = math.floor((28.5 - row['end_lat'])/0.25)
        difx = abs(startgridx - endgridx)
        dify = abs(startgridy - endgridy)

        startgrid_x.append(startgridx)
        startgrid_y.append(startgridy)
        endgrid_x.append(endgridx)
        endgrid_y.append(endgridy)
        dif_x.append(difx)
        dif_y.append(dify)

    grid = pd.DataFrame({"startgrid_x":startgrid_x,
                        "startgrid_y":startgrid_y,
                        "endgrid_x":endgrid_x,
                        "endgrid_y":endgrid_y,
                        "dif_x":dif_x,
                        "dif_y":dif_y})

    with_grid = pd.concat([aggregate, grid], axis = 1)

    # Filter those data not within one or two adjacent grid
    with_grid.drop(with_grid[(with_grid['dif_x']>1) | (with_grid['dif_y']>1)].index,axis=0, inplace = True)

    # AGGREGATE BY GRID
    aggregate_value = ['n_crisis','difference','baseline']
    aggregate2 = with_grid.groupby(['country','year','month','day','startgrid_x','startgrid_y','endgrid_x','endgrid_y'])[aggregate_value].sum()
    aggregate2 = aggregate2.reset_index() # return dataframe of groupby result
    date = aggregate2['year'].str.cat(aggregate2['month'],sep='-').str.cat(aggregate2['day'],sep='-')
    aggregate3 = aggregate2.drop(['year','month','day'], axis = 1)
    aggregate3 = pd.concat([aggregate3, date], axis = 1)
    aggregate3.rename(columns={"year":"date"}, inplace=True)

    aggregate3.to_csv("aggtemp.csv", mode='a', header=True, index=False)

    # Auto rename
    os.rename("aggtemp.csv","Agg" + country + aggregate2.iloc[0,1] + aggregate2.iloc[0,2] + ".csv")
    print('Aggregate Complete')
    
    # assign value to grid
    df3 = aggregate3
    df3.eval('movement_value = n_crisis / 2' , inplace=True)
    df4 = df3.drop(["endgrid_x","endgrid_y"], axis = 1)
    df5 = df3.drop(["startgrid_x","startgrid_y"], axis = 1)
    df4.rename(columns={"startgrid_x":"grid_x", "startgrid_y":"grid_y"}, inplace = True)
    df5.rename(columns={"endgrid_x":"grid_x", "endgrid_y":"grid_y"}, inplace = True)
    Cal_grid = pd.concat([df4, df5], axis = 0)
    Cal_grid = Cal_grid.groupby(['country','date','grid_x','grid_y'])[['n_crisis','difference','baseline','movement_value']].sum()
    Cal_grid = Cal_grid.reset_index()

    # Assign grid coordinates
    Cal_grid['grid_lon'] = Cal_grid['grid_x'].map(lambda x: 92.375 + 0.25*x )
    Cal_grid['grid_lat'] = Cal_grid['grid_y'].map(lambda y: 28.375 - 0.25*y )
    filepath = "E:\GRADUATE STUDY\GE5219\Data\Study Area\SEA_GRID.csv"
#     coor = pd.read_csv(filepath)
#     Cal_grid = Cal_grid.merge(coor,on=['grid_x','grid_y'],how='left')
    Cal_grid.to_csv("gridtemp.csv", mode='a', header=True, index=False)
    
    # Auto rename
    os.rename("gridtemp.csv","Grid" + country + aggregate2.iloc[0,1] + aggregate2.iloc[0,2] + ".csv")
    print('Generate Grid Complete')

### Auto Process

In [ ]:
def Auto_Process(folder_path, country, country_iso2):
    folder_path = os.path.abspath(folder_path)
    for year in range(2020,2023):
        for month in range(1,13):
            if year==2020 and month<6:
                continue
            elif year==2022 and month>2:
                break
            else:
                folder = "{}{}{:0>2d}".format(country,year,month)
                path = folder_path + "\\" + folder
                Process_File(path, country_iso2)
    print("Finish processing {}".format(country))

In [ ]:
Auto_Process("E:\GRADUATE STUDY\GE5219\Data\Movement\Processing", "Brunei", "BN")

# Apple Mobility

In [ ]:
# READ FILE
filepath = r"E:\GRADUATE STUDY\GE5219\Data\Movement\Apple Mobility\applemobilitytrends-2022-03-10.csv"
df = pd.read_csv(filepath)
df

In [ ]:
regions = ['Singapore','Malaysia','Thailand','Laos','Vietnam','Philippines','Myanmar','Cambodia','Brunei']
df2 = df.loc[df['region'].isin(regions)]

In [ ]:
dfT = pd.DataFrame(df2.values.T, index=df2.columns, columns=df2.index)

In [ ]:
dfT.drop(index = ["geo_type","alternative_name","sub-region","country"], inplace = True)
dfT.drop(dfT.tail(10).index,inplace=True) 

In [ ]:
dfT

In [ ]:
dfT.to_csv("applemobility.csv", header = None)

In [ ]:
filepath = "applemobility.csv"
reading = pd.read_csv(filepath)
reading

In [ ]:
reading.describe(percentiles=[.2,.75, .8],include='all')